# Construcción de la base de datos

### 1) *Importar los datos*

In [73]:
import wbdata

In [2]:
wb_regions = ['Arab World', 'Caribbean small states',
              'Central Europe and the Baltics', 'Early-demographic dividend',
              'East Asia & Pacific', 'East Asia & Pacific (excluding high income)',
              'East Asia & Pacific (IDA & IBRD countries)',
              'Euro area', 'Europe & Central Asia',
              'Europe & Central Asia (excluding high income)',
              'Europe & Central Asia (IDA & IBRD countries)',
              'European Union',
              'Fragile and conflict affected situations',
              'Heavily indebted poor countries (HIPC)',
              'High income',
              'IBRD only',
              'IDA & IBRD total',
              'IDA blend',
              'IDA only',
              'IDA total',
              'Late-demographic dividend',
              'Latin America & Caribbean',
              'Latin America & Caribbean (excluding high income)',
              'Latin America & the Caribbean (IDA & IBRD countries)',
              'Least developed countries',
              'Least developed countries: UN classification',
              'Low & middle income',
              'Low income',
              'Lower middle income',
              'Middle East & North Africa',
              'Middle East & North Africa (excluding high income)',
              'Middle East & North Africa (IDA & IBRD countries)',
              'Middle income',
              'North America',
              'Not classified',
              'OECD members',
              'Other small states',
              'Pacific island small states',
              'Post-demographic dividend',
              'Pre-demographic dividend',
              'Small states',
              'South Asia',
              'South Asia (IDA & IBRD)',
              'Sub-Saharan Africa',
              'Sub-Saharan Africa (excluding high income)',
              'Sub-Saharan Africa (IDA & IBRD countries)',
              'Upper middle income',
              'World']

In [3]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pandas as pd
import numpy as np
import statsmodels.api as sm

### 2) *Definir  y limpiar las Variables*

In [4]:
Governmentdebt_raw = wbdata.get_data("GC.DOD.TOTL.GD.ZS")

In [5]:
Inflation_raw = wbdata.get_data("FP.CPI.TOTL.ZG")

In [6]:
Expense_raw = wbdata.get_data("GC.XPN.TOTL.GD.ZS")

In [7]:
Tax_raw = wbdata.get_data("GC.TAX.TOTL.GD.ZS")

In [8]:
Reserves_raw = wbdata.get_data("FI.RES.TOTL.CD")

In [9]:
Officialexchange_raw = wbdata.get_data("PA.NUS.FCRF")

In [10]:
Interestrate_raw = wbdata.get_data("FR.INR.RINR")

In [11]:
Unemployment_raw = wbdata.get_data("SL.UEM.TOTL.ZS")

In [65]:
def wb_clean_year_data(wb_raw, year):
    rv = {}
    for X in wb_raw:
        date = X['date']
        country = X['country']['value']
        if date == year and country not in wb_regions:
            try:
                value = float(X['value'])
                rv[country] = value
            except:
                pass
    return rv

In [66]:
def wb_dict_to_dataframe(dict_clean, nombre_variable, año):
    df = pd.DataFrame({'pais': list(dict_clean.keys()),
                       nombre_variable: list(dict_clean.values()),
                       'año': [año] * len(dict_clean)})
    return df

### 3) *Obtenemos las variables limpias*

In [67]:
variables=['GC.DOD.TOTL.GD.ZS','FP.CPI.TOTL.ZG','GC.XPN.TOTL.GD.ZS','GC.TAX.TOTL.GD.ZS','FI.RES.TOTL.CD','PA.NUS.FCRF','FR.INR.RINR','SL.UEM.TOTL.ZS']

In [68]:
Governmentdebt_raw = wbdata.get_data("GC.DOD.TOTL.GD.ZS")
Inflation_raw = wbdata.get_data("FP.CPI.TOTL.ZG")
Expense_raw = wbdata.get_data("GC.XPN.TOTL.GD.ZS")
Tax_raw = wbdata.get_data("GC.TAX.TOTL.GD.ZS")
Reserves_raw = wbdata.get_data("FI.RES.TOTL.CD")
Officialexchange_raw = wbdata.get_data("PA.NUS.FCRF")
Interestrate_raw = wbdata.get_data("FR.INR.RINR")
Unemployment_raw = wbdata.get_data("SL.UEM.TOTL.ZS")

In [69]:
raw_lists = {'Deuda': Governmentdebt_raw,
             'Inflacion': Inflation_raw,
             'Gasto público': Expense_raw,
             'Tasa fiscal': Tax_raw,
             'Reservas':Reserves_raw,
             'Tipo de cambio':Officialexchange_raw,
             'Tasa de interes':Interestrate_raw,}
            

In [70]:
for año in range(1989, 2011):
    dfs_año = []
    for k, v in raw_lists.items():
        v_clean = wb_clean_year_data(v, str(año))
        df = wb_dict_to_dataframe(v_clean, k, str(año))
        dfs_año.append(df)
    if año == 1989:
        for i, df in enumerate(dfs_año):
            if i == 0:
                df_año = df.merge(dfs_año[i + 1], on=['pais', 'año'])
            else:
                try:
                    df_año = df_año.merge(dfs_año[i + 1], on=['pais', 'año'])
                except:
                    pass
    else:
        for i, df in enumerate(dfs_año):
            if i == 0:
                df_otros_años = df.merge(dfs_año[i + 1], on=['pais', 'año'])
            else:
                try:
                    df_otros_años = df_otros_años.merge(dfs_año[i + 1], on=['pais', 'año'])
                except:
                    pass
        df_año = df_año.append(df_otros_años, ignore_index=True,)

In [71]:
df_año['año'].unique()

array(['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010'], dtype=object)

In [72]:
df_año ['pais'].value_counts()

United States          22
Mauritius              21
United Kingdom         21
Bhutan                 21
India                  21
                       ..
Trinidad and Tobago     2
Korea, Rep.             2
Malawi                  2
Egypt, Arab Rep.        1
Zambia                  1
Name: pais, Length: 63, dtype: int64

### 4) *Balanceamos la base de datos*

In [24]:
paises_buenos = []
for country, counts in dict(df_año['pais'].value_counts()).items():
    if counts >= 21:
        paises_buenos.append(country)

In [25]:
df_año_paises_buenos = df_año.loc[df_año['pais'].isin(paises_buenos)]

In [26]:
df_año_paises_buenos = df_año_paises_buenos.loc[df_año_paises_buenos['año']!="1989"]

In [27]:
df_año_paises_buenos ['pais'].value_counts()

Bhutan            21
India             21
Jamaica           21
United Kingdom    21
United States     21
Iceland           21
Australia         21
Singapore         21
Mauritius         21
Name: pais, dtype: int64

In [43]:
df_año_paises_buenos.head(100)

,pais,Deuda,año,Inflacion,Gasto público,Tasa fiscal,Reservas,Tipo de cambio,Tasa de interes
1,Australia,12.253160,1990,7.333022,21.861559,22.476879,1.931874e+10,1.281057,9.669791
4,Bhutan,51.188019,1990,10.000000,20.557124,4.413943,8.883421e+07,17.505325,8.789378
7,Iceland,29.869468,1990,15.510722,29.377902,23.472225,4.549824e+08,58.283775,0.878499
8,India,50.783254,1990,8.971233,15.484217,9.993591,5.637446e+09,17.503500,5.269527
10,Jamaica,138.741328,1990,21.960175,22.642856,23.486431,1.681553e+08,7.184025,4.302951
...,...,...,...,...,...,...,...,...,...
284,Mauritius,32.117443,2000,4.199288,20.092293,16.953664,9.143661e+08,26.249558,16.788430
291,Singapore,83.842292,2000,1.361624,15.716732,14.855789,8.108498e+10,1.723963,1.897404
297,United Kingdom,42.607454,2000,1.182956,32.251054,26.072901,5.093939e+10,0.660931,4.028270
298,United States,33.263120,2000,3.376857,17.924094,12.969032,1.283995e+11,1.000000,6.844844


In [29]:
df_año_paises_buenos.to_excel('base_final.xlsx')

### 5) *Agregamos la variable dummy*

In [35]:
Base_final = pd.read_excel('/Users/joaco/Downloads/trabajo_econometria_up_5-master 4/base_final_1.xlsx')

In [51]:
Base_final.head(100)

,Pais,Año,Deuda,Inflacion,Gasto público,Tasa fiscal,Reservas,Tipo de cambio,Defaults prev,Default previo_c_t (3 años)
0,Australia,1990,12.253160,7.333022,21.861559,22.476879,1.931874e+10,1.281057,0,0
1,Bhutan,1990,51.188019,10.000000,20.557124,4.413943,8.883421e+07,17.505325,1,0
2,Guatemala,1990,24.480685,41.221867,9.131265,6.865212,3.618105e+08,4.485758,1,1
3,Iceland,1990,29.869468,15.510722,29.377902,23.472225,4.549824e+08,58.283775,0,0
4,India,1990,50.783254,8.971233,15.484217,9.993591,5.637446e+09,17.503500,0,0
...,...,...,...,...,...,...,...,...,...,...
95,Tunisia,1996,55.296518,3.725145,28.016448,19.837619,1.977688e+09,0.973408,1,1
96,United Kingdom,1996,43.628347,2.851782,33.857007,23.134875,4.670002e+10,0.640958,0,0
97,United States,1996,46.741769,2.931204,20.005396,11.373915,1.606602e+11,1.000000,0,0
98,Australia,1997,20.911558,0.224888,23.864424,22.430942,1.758826e+10,1.347380,0,0


# Media y varianza de cada variable

In [52]:
Base_final.columns

Index(['Pais', 'Año', 'Deuda', 'Inflacion', 'Gasto público', 'Tasa fiscal',
       'Reservas', 'Tipo de cambio', 'Defaults prev',
       'Default previo_c_t (3 años)'],
      dtype='object')

In [76]:
import statistics as stats
from scipy import stats

### Deuda

In [50]:
Base_final['Deuda'].describe()

count    294.000000
mean      50.851689
std       33.307938
min        0.018398
25%       26.459870
50%       44.853644
75%       64.231988
max      232.832211
Name: Deuda, dtype: float64

### Inflación

In [53]:
Base_final['Inflacion'].describe()

count    294.000000
mean       5.644940
std        7.337135
min      -18.108630
25%        2.205611
50%        3.890281
75%        6.830412
max       77.296593
Name: Inflacion, dtype: float64

### Gasto Público

In [56]:
Base_final['Gasto público'].describe()

count    294.000000
mean      21.875636
std        7.293717
min        7.562050
25%       16.466839
50%       20.175852
75%       27.094002
max       42.283512
Name: Gasto público, dtype: float64

### Tasa Fiscal

In [58]:
Base_final['Tasa fiscal'].describe()

count    294.000000
mean      16.127629
std        6.051511
min        4.413943
25%       10.261597
50%       16.078596
75%       21.830268
max       27.226929
Name: Tasa fiscal, dtype: float64

### Reservas

In [59]:
Base_final['Reservas'].describe()

count    2.940000e+02
mean     4.440132e+10
std      7.065982e+10
min      8.512000e+07
25%      1.147301e+09
50%      1.052933e+10
75%      6.015286e+10
max      4.889285e+11
Name: Reservas, dtype: float64

### Tipo de Cambio

In [60]:
Base_final['Tipo de cambio'].describe()

count    294.000000
mean      19.567441
std       24.735308
min        0.499772
25%        1.094097
50%        3.209600
75%       35.425997
max      123.638381
Name: Tipo de cambio, dtype: float64

### Defaults previos

In [62]:
Base_final['Defaults prev'].describe()

count    294.000000
mean       0.302721
std        0.460219
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Defaults prev, dtype: float64

### Defaults previos rezago de 3 años

In [64]:
Base_final['Default previo_c_t (3 años)'].describe()

count    294.000000
mean       0.367347
std        0.482904
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Default previo_c_t (3 años), dtype: float64

### *Media y varianza de todas las variables*

In [78]:
Base_final.mean()

Año                            2.000000e+03
Deuda                          5.085169e+01
Inflacion                      5.644940e+00
Gasto público                  2.187564e+01
Tasa fiscal                    1.612763e+01
Reservas                       4.440132e+10
Tipo de cambio                 1.956744e+01
Defaults prev                  3.027211e-01
Default previo_c_t (3 años)    3.673469e-01
dtype: float64

In [79]:
Base_final.var()

Año                            3.679181e+01
Deuda                          1.109419e+03
Inflacion                      5.383355e+01
Gasto público                  5.319830e+01
Tasa fiscal                    3.662078e+01
Reservas                       4.992810e+21
Tipo de cambio                 6.118355e+02
Defaults prev                  2.118014e-01
Default previo_c_t (3 años)    2.331964e-01
dtype: float64

In [80]:
np.mean(Base_final)

Año                            2.000000e+03
Deuda                          5.085169e+01
Inflacion                      5.644940e+00
Gasto público                  2.187564e+01
Tasa fiscal                    1.612763e+01
Reservas                       4.440132e+10
Tipo de cambio                 1.956744e+01
Defaults prev                  3.027211e-01
Default previo_c_t (3 años)    3.673469e-01
dtype: float64

In [82]:
np.var(Base_final, 0)

Año                            3.666667e+01
Deuda                          1.105645e+03
Inflacion                      5.365044e+01
Gasto público                  5.301735e+01
Tasa fiscal                    3.649622e+01
Reservas                       4.975828e+21
Tipo de cambio                 6.097544e+02
Defaults prev                  2.110810e-01
Default previo_c_t (3 años)    2.324032e-01
dtype: float64